<a href="https://colab.research.google.com/github/AkhdanFirdaus/bmn-model/blob/main/final_ta_klasifikasi_teks_bert_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Klasifikasi Teks Laporan Mobil Dinas dengan BERT

# 4.3 Data Preparation

In [1]:
# Mount Drive

from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
location_invoice = '/content/drive/MyDrive/Kuliah/TA Akhdan Musyaffa Firdaus/Bimbingan - Akhdan Musyaffa Firdaus/Dataset/Raw Data/raw_invoice.csv'
location_sukucadang = '/content/drive/MyDrive/Kuliah/TA Akhdan Musyaffa Firdaus/Bimbingan - Akhdan Musyaffa Firdaus/Dataset/Raw Data/raw_sukucadang.csv'
location_kerusakan = '/content/drive/MyDrive/Kuliah/TA Akhdan Musyaffa Firdaus/Bimbingan - Akhdan Musyaffa Firdaus/Dataset/Raw Data/raw_kategori_kerusakan.csv'
location_laporan = '/content/drive/MyDrive/Kuliah/TA Akhdan Musyaffa Firdaus/Bimbingan - Akhdan Musyaffa Firdaus/Dataset/Raw Data/raw_laporan.csv'

## 4.3.1 Pemilahan Data

In [3]:
import numpy as np
import pandas as pd

In [4]:
data_invoice = pd.read_csv(location_invoice)[['Uraian Pekerjaan', 'Harga Satuan']]
data_sukucadang =  pd.read_csv(location_sukucadang)[['Komponen', 'Keterangan']]
data_kerusakan = pd.read_csv(location_kerusakan)[['Kerusakan', 'Keterangan', 'Kategori']]
data_laporan = pd.read_csv(location_laporan)[['Laporan', 'Masalah']]

In [5]:
print('Invoice = ', data_invoice.shape)
print('Sukucadang = ', data_sukucadang.shape)
print('Kerusakan = ', data_kerusakan.shape)
print('Laporan = ', data_laporan.shape)

Invoice =  (526, 2)
Sukucadang =  (210, 2)
Kerusakan =  (11, 3)
Laporan =  (138, 2)


### Preview

In [6]:
data_invoice.head()

,Uraian Pekerjaan,Harga Satuan
0,Shell HX-6 10/40 4Ltr,"420,000"
1,Filter Oli YZZE1,"60,000"
2,Filter Udara Avanza VVTi,"175,000"
3,Service Rem Depan + Belakang,"300,000"
4,Tune Up Injection Cleaner,"350,000"


In [7]:
data_sukucadang.head()

,Komponen,Keterangan
0,Shell HX-6 10/40 4Ltr,Oli mesin untuk kendaraan roda empat dengan b...
1,Filter Oli YZZE1,Filter oli untuk kendaraan roda empat yang di...
2,Filter Udara Avanza VVTi,Filter udara kualitas tinggi untuk kendaraan ...
3,Service Rem Depan + Belakang,Layanan perawatan rem depan dan belakang pada...
4,Tune Up Injection Cleaner,Layanan tune up dengan teknologi injection Cl...


In [8]:
data_kerusakan.head()

,Kerusakan,Keterangan,Kategori
0,Masalah Sistem Knalpot,Sistem pembuangan bertanggung jawab untuk meng...,Filter Udara;Oli;Bahan Bakar;Cleaner;Pemasangan
1,Masalah Transmisi,Masalah transmisi kendaraan roda empat adalah ...,Oli;Transmisi;Pemasangan;Kopling;Joint;Bearing...
2,Masalah Suspensi,Suspensi kendaraan roda empat adalah kumpulan ...,Shockbreaker;Spooring;Balancing;Hidrolik;Pemas...
3,Gangguan Listrik,Gangguan listrik pada kendaraan roda empat ter...,Lampu;Listrik;Pemasangan
4,Masalah Sistem Bahan Bakar,Masalah sistem bahan bakar kendaraan roda empa...,Bahan Bakar;Cleaner;Pemasangan;Kruk As


In [9]:
data_laporan.head()

,Laporan,Masalah
0,Saya melihat indikator check engine menyala di...,Masalah Mesin
1,"Saat saya menginjak rem, terdengar suara berde...",Kegagalan Rem
2,Saya merasakan getaran yang tidak normal saat ...,Masalah Kemudi;Masalah Ban
3,"Mobil saya stir nya tidak stabil, lalu pada sa...",Gangguan Listrik;Masalah Kemudi;Masalah Mesin
4,kaca mobil tidak bisa bergerak,"Kerusakan Aksesoris Interior, Eksterior"


## 4.3.2 Pembersihan Data

In [10]:
sample_texts = data_laporan.head(3)['Laporan']
print(sample_texts[0])
print(sample_texts[1])
print(sample_texts[2])

Saya melihat indikator check engine menyala di dashboard mobil saya.
Saat saya menginjak rem, terdengar suara berdecit yang tidak biasa.
Saya merasakan getaran yang tidak normal saat mengemudi di kecepatan tinggi.


### Casefolding

In [11]:
def casefolding(val):
  return str(val).lower()

In [12]:
hasil_casefolding = [casefolding(sample) for sample in sample_texts]
print(hasil_casefolding[0])
print(hasil_casefolding[1])
print(hasil_casefolding[2])

saya melihat indikator check engine menyala di dashboard mobil saya.
saat saya menginjak rem, terdengar suara berdecit yang tidak biasa.
saya merasakan getaran yang tidak normal saat mengemudi di kecepatan tinggi.


In [13]:
data_invoice = data_invoice.applymap(casefolding)
data_sukucadang = data_sukucadang.applymap(casefolding)
data_kerusakan = data_kerusakan.applymap(casefolding)
data_laporan = data_laporan.applymap(casefolding)

### Cleaning

In [14]:
import re

def cleaning(val):
  # Membersihkan Whitespace
  val = re.sub(r'\s+', ' ', val)

  # Hanya Mengambil karakter alfanumerik
  val = re.sub("[^a-zA-Z0-9]", " ", val)

  return val

In [15]:
hasil_cleaning = [cleaning(sample) for sample in hasil_casefolding]
print(hasil_cleaning[0])
print(hasil_cleaning[1])
print(hasil_cleaning[2])

saya melihat indikator check engine menyala di dashboard mobil saya 
saat saya menginjak rem  terdengar suara berdecit yang tidak biasa 
saya merasakan getaran yang tidak normal saat mengemudi di kecepatan tinggi 


In [16]:
data_invoice = data_invoice.applymap(cleaning)
data_sukucadang = data_sukucadang.applymap(cleaning)
data_kerusakan = data_kerusakan.applymap(cleaning)
data_laporan = data_laporan.applymap(cleaning)

### Stemming

In [17]:
pip install Sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.3 MB/s eta 0:00:00


In [18]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

stemmer = StemmerFactory().create_stemmer()

def stemming(val):
  return stemmer.stem(str(val))

In [19]:
hasil_stemming = [stemming(sample) for sample in hasil_cleaning]
print(hasil_stemming[0])
print(hasil_stemming[1])
print(hasil_stemming[2])

saya lihat indikator check engine nyala di dashboard mobil saya
saat saya injak rem dengar suara decit yang tidak biasa
saya rasa getar yang tidak normal saat kemudi di cepat tinggi


In [20]:
data_invoice = data_invoice.applymap(stemming)
data_sukucadang = data_sukucadang.applymap(stemming)
data_kerusakan = data_kerusakan.applymap(stemming)
data_laporan = data_laporan.applymap(stemming)

### Stopword Removal

In [21]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

stopword_remover = StopWordRemoverFactory().create_stop_word_remover()

def stopwordremove(val):
  return stopword_remover.remove(str(val))

In [22]:
hasil_stopword_remove = [stopwordremove(sample) for sample in hasil_stemming]
print(hasil_stopword_remove[0])
print(hasil_stopword_remove[1])
print(hasil_stopword_remove[2])

lihat indikator check engine nyala dashboard mobil
saya injak rem dengar suara decit tidak biasa
rasa getar tidak normal kemudi cepat tinggi


In [23]:
data_invoice = data_invoice.applymap(stopwordremove)
data_sukucadang = data_sukucadang.applymap(stopwordremove)
data_kerusakan = data_kerusakan.applymap(stopwordremove)
data_laporan = data_laporan.applymap(stopwordremove)

### Filtering

In [24]:
data_invoice = data_invoice.drop_duplicates(subset=['Uraian Pekerjaan'])
data_sukucadang = data_sukucadang.drop_duplicates(subset=['Komponen'])
data_kerusakan = data_kerusakan.drop_duplicates(subset=['Kerusakan'])
data_laporan = data_laporan.drop_duplicates(subset=['Laporan'])

In [25]:
print('Invoice = ', data_invoice.shape)
print('Sukucadang = ', data_sukucadang.shape)
print('Kerusakan = ', data_kerusakan.shape)
print('Laporan = ', data_laporan.shape)

Invoice =  (199, 2)
Sukucadang =  (200, 2)
Kerusakan =  (11, 3)
Laporan =  (121, 2)


### Preview

In [26]:
data_invoice

,Uraian Pekerjaan,Harga Satuan
0,shell hx 6 10 40 4ltr,420 000
1,filter oli yzze1,60 000
2,filter udara avanza vvti,175 000
3,service rem depan belakang,300 000
4,tune up injection cleaner,350 000
...,...,...
520,bohlam forglamp h11 55watt,75 000
521,joint copel,650 000
522,karet stabilizer r l,325 000
523,sikring 20 ah,20 000


In [27]:
data_sukucadang

,Komponen,Keterangan
0,shell hx 6 10 40 4ltr,oli mesin kendara roda empat bobot 4 liter fun...
1,filter oli yzze1,filter oli kendara roda empat guna nyaring kot...
2,filter udara avanza vvti,filter udara kualitas tinggi kendara roda empa...
3,service rem depan belakang,layan awat rem depan belakang kendara roda empat
4,tune up injection cleaner,layan tune up teknologi injection cleaner bers...
...,...,...
205,joint copel,joint copel kendara roda empat
206,karet stabilizer r l,karet stabilizer kendara roda empat kanan kiri
207,sikring 20 ah,sikring kapasitas 20 ah kendara roda empat
208,mobil derek towing,layan derek kendara roda empat


In [28]:
data_kerusakan.head()

,Kerusakan,Keterangan,Kategori
0,masalah sistem knalpot,sistem buang tanggung jawab hilang gas limbah ...,filter udara oli bahan bakar cleaner pasang
1,masalah transmisi,masalah transmisi kendara roda empat kondisi m...,oli transmisi pasang kopling joint bearing pas...
2,masalah suspensi,suspensi kendara roda empat kumpul komponen fu...,shockbreaker spooring balancing hidrolik pasang
3,ganggu listrik,ganggu listrik kendara roda empat jadi ada mas...,lampu listrik pasang
4,masalah sistem bahan bakar,masalah sistem bahan bakar kendara roda empat ...,bahan bakar cleaner pasang kruk as


In [29]:
data_laporan.head()

,Laporan,Masalah
0,lihat indikator check engine nyala dashboard m...,masalah mesin
1,saya injak rem dengar suara decit tidak biasa,gagal rem
2,rasa getar tidak normal kemudi cepat tinggi,masalah kemudi masalah ban
3,mobil stir nya stabil lalu menstater kadang ka...,ganggu listrik masalah kemudi masalah mesin
4,kaca mobil bisa gerak,rusa aksesoris interior eksterior


## 4.3.3 Konstruksi Data

### Penentuan Fitur

Akan dibuat sebuah dataset untuk mendeteksi konteks
1. Menggabungkan data invoice dan suku cadang
2. menggabungkan beberapa atribut untuk memperkuat konteks:
- 'Uraian Pekerjaan', 'Kategori', 'Total'
- 'Komponen', 'Keterangan'
- 'Kerusakan', 'Keterangan', 'Kategori'
- 'Laporan', 'Kategori'
3. menggabungkan data diatas dengan sukucadang berdasar pekerjaan

dataset 1:
dataset_1 = konteks,

dataset 2:

### Pemilihan Atribut

In [30]:
data_invoice_construct = data_invoice.copy().rename(columns={
    'Uraian Pekerjaan': 'suku_cadang',
    'Harga Satuan': 'biaya'
})

data_sukucadang_construct = data_sukucadang.copy().rename(columns={
    'Komponen': 'suku_cadang',
    'Keterangan': 'kategori'
})

# Menggabungkan data invoice dan suku cadang
dataset_1 = pd.merge(
    data_invoice_construct,
    data_sukucadang_construct,
    on='suku_cadang',
    how='left'
)

dataset_1['konteks'] = ''

dataset_1.head()

,suku_cadang,biaya,kategori,konteks
0,shell hx 6 10 40 4ltr,420 000,oli mesin kendara roda empat bobot 4 liter fun...,
1,filter oli yzze1,60 000,filter oli kendara roda empat guna nyaring kot...,
2,filter udara avanza vvti,175 000,filter udara kualitas tinggi kendara roda empa...,
3,service rem depan belakang,300 000,layan awat rem depan belakang kendara roda empat,
4,tune up injection cleaner,350 000,layan tune up teknologi injection cleaner bers...,


In [31]:
dataset_2 = data_laporan.copy().rename(columns={
    'Laporan': 'konteks',
    'Masalah': 'masalah'
})
dataset_2['suku_cadang'] = ''
dataset_2['kategori'] = ''
dataset_2['biaya'] = ''
dataset_2 = dataset_2[['suku_cadang', 'kategori', 'biaya', 'konteks', 'masalah']]
dataset_2.head()

,suku_cadang,kategori,biaya,konteks,masalah
0,,,,lihat indikator check engine nyala dashboard m...,masalah mesin
1,,,,saya injak rem dengar suara decit tidak biasa,gagal rem
2,,,,rasa getar tidak normal kemudi cepat tinggi,masalah kemudi masalah ban
3,,,,mobil stir nya stabil lalu menstater kadang ka...,ganggu listrik masalah kemudi masalah mesin
4,,,,kaca mobil bisa gerak,rusa aksesoris interior eksterior


## 4.3.4 Pelabelan Data

### Penentuan Label

In [32]:
# Ekstraksi label kerusakan dan level dari dataset
LABELS = data_kerusakan['Kerusakan'].tolist()
print("Label kerusakan = ", LABELS)

Label kerusakan =  ['masalah sistem knalpot', 'masalah transmisi', 'masalah suspensi', 'ganggu listrik', 'masalah sistem bahan bakar', 'masalah kemudi', 'masalah ban', 'masalah mesin', 'gagal rem', 'masalah sistem dingin', 'rusa aksesoris interior eksterior']


### Penerapan Label

In [33]:
# Menambahkan Kolom Label
dataset_1_labeled = dataset_1.copy()
dataset_1_labeled[LABELS] = 0
dataset_1_labeled.head()

,suku_cadang,biaya,kategori,konteks,masalah sistem knalpot,masalah transmisi,masalah suspensi,ganggu listrik,masalah sistem bahan bakar,masalah kemudi,masalah ban,masalah mesin,gagal rem,masalah sistem dingin,rusa aksesoris interior eksterior
0,shell hx 6 10 40 4ltr,420 000,oli mesin kendara roda empat bobot 4 liter fun...,,0,0,0,0,0,0,0,0,0,0,0
1,filter oli yzze1,60 000,filter oli kendara roda empat guna nyaring kot...,,0,0,0,0,0,0,0,0,0,0,0
2,filter udara avanza vvti,175 000,filter udara kualitas tinggi kendara roda empa...,,0,0,0,0,0,0,0,0,0,0,0
3,service rem depan belakang,300 000,layan awat rem depan belakang kendara roda empat,,0,0,0,0,0,0,0,0,0,0,0
4,tune up injection cleaner,350 000,layan tune up teknologi injection cleaner bers...,,0,0,0,0,0,0,0,0,0,0,0


In [34]:
# Menambahkan Kolom Label
dataset_2_labeled = dataset_2.copy()
dataset_2_labeled[LABELS] = 0
dataset_2_labeled.head()

,suku_cadang,kategori,biaya,konteks,masalah,masalah sistem knalpot,masalah transmisi,masalah suspensi,ganggu listrik,masalah sistem bahan bakar,masalah kemudi,masalah ban,masalah mesin,gagal rem,masalah sistem dingin,rusa aksesoris interior eksterior
0,,,,lihat indikator check engine nyala dashboard m...,masalah mesin,0,0,0,0,0,0,0,0,0,0,0
1,,,,saya injak rem dengar suara decit tidak biasa,gagal rem,0,0,0,0,0,0,0,0,0,0,0
2,,,,rasa getar tidak normal kemudi cepat tinggi,masalah kemudi masalah ban,0,0,0,0,0,0,0,0,0,0,0
3,,,,mobil stir nya stabil lalu menstater kadang ka...,ganggu listrik masalah kemudi masalah mesin,0,0,0,0,0,0,0,0,0,0,0
4,,,,kaca mobil bisa gerak,rusa aksesoris interior eksterior,0,0,0,0,0,0,0,0,0,0,0


In [35]:
# Fungsi Labeling
def labeling(val, key):
  labels = {}
  # update_konteks = konteks

  for index, row in data_kerusakan.iterrows():
    array1 = row[key].split(' ')
    array2 = val.split(' ')

    kerusakan = row['Kerusakan']
    konteks_tambahan = row['Keterangan']

    kondisi = 1 if any(x in array2 for x in array1) else 0

    print(kondisi, ' - ', row[key], ' - ', val)
    # update_konteks += ' ' + konteks_tambahan if kondisi == 1 else ""
    labels.update({kerusakan: kondisi})

  # labels.update({'konteks': update_konteks})
  return labels

# Fungsi Labeling
def labeling2(val, key):
  labels = {}

  for index, row in data_kerusakan.iterrows():
    array1 = row[key].split(' ')
    array2 = val.replace('masalah', '') if 'masalah' in val else val

    kerusakan = row['Kerusakan']
    konteks_tambahan = row['Keterangan']

    kondisi = 1 if any(x in array2 for x in array1) else 0

    print(kondisi, ' - ', row[key], ' - ', val)
    labels.update({kerusakan: kondisi})

  return labels

In [36]:
for index, row in dataset_1_labeled.iterrows():
  kategori = row['kategori']
  update_value = labeling(kategori, 'Kategori')
  dataset_1_labeled.loc[index, list(update_value.keys())] = list(update_value.values())

1  -  filter udara oli bahan bakar cleaner pasang  -  oli mesin kendara roda empat bobot 4 liter fungsi bagai lumas mesin mobil teknologi sintetis bantu lindung mesin kendara dari kotor sisa bakar aus beri performa maksimal tahan lama oli telah teknologi active cleansing technology mampu jaga mesin tetap bersih mesin selalu asa baru itu oli dapat kurang gejala knocking mesin turbocharger
1  -  oli transmisi pasang kopling joint bearing pasang cleaner  -  oli mesin kendara roda empat bobot 4 liter fungsi bagai lumas mesin mobil teknologi sintetis bantu lindung mesin kendara dari kotor sisa bakar aus beri performa maksimal tahan lama oli telah teknologi active cleansing technology mampu jaga mesin tetap bersih mesin selalu asa baru itu oli dapat kurang gejala knocking mesin turbocharger
0  -  shockbreaker spooring balancing hidrolik pasang  -  oli mesin kendara roda empat bobot 4 liter fungsi bagai lumas mesin mobil teknologi sintetis bantu lindung mesin kendara dari kotor sisa bakar aus

In [37]:
for index, row in dataset_2_labeled.iterrows():
  masalah = row['masalah']
  update_value = labeling2(masalah, 'Kerusakan')
  dataset_2_labeled.loc[index, list(update_value.keys())] = list(update_value.values())

0  -  masalah sistem knalpot  -  masalah mesin
0  -  masalah transmisi  -  masalah mesin
0  -  masalah suspensi  -  masalah mesin
0  -  ganggu listrik  -  masalah mesin
0  -  masalah sistem bahan bakar  -  masalah mesin
0  -  masalah kemudi  -  masalah mesin
0  -  masalah ban  -  masalah mesin
1  -  masalah mesin  -  masalah mesin
0  -  gagal rem  -  masalah mesin
0  -  masalah sistem dingin  -  masalah mesin
0  -  rusa aksesoris interior eksterior  -  masalah mesin
0  -  masalah sistem knalpot  -  gagal rem
0  -  masalah transmisi  -  gagal rem
0  -  masalah suspensi  -  gagal rem
0  -  ganggu listrik  -  gagal rem
0  -  masalah sistem bahan bakar  -  gagal rem
0  -  masalah kemudi  -  gagal rem
0  -  masalah ban  -  gagal rem
0  -  masalah mesin  -  gagal rem
1  -  gagal rem  -  gagal rem
0  -  masalah sistem dingin  -  gagal rem
0  -  rusa aksesoris interior eksterior  -  gagal rem
0  -  masalah sistem knalpot  -  masalah kemudi masalah ban
0  -  masalah transmisi  -  masalah kemudi

In [38]:
dataset_1_labeled.shape

(199, 15)

In [39]:
dataset_1_labeled.head()

,suku_cadang,biaya,kategori,konteks,masalah sistem knalpot,masalah transmisi,masalah suspensi,ganggu listrik,masalah sistem bahan bakar,masalah kemudi,masalah ban,masalah mesin,gagal rem,masalah sistem dingin,rusa aksesoris interior eksterior
0,shell hx 6 10 40 4ltr,420 000,oli mesin kendara roda empat bobot 4 liter fun...,,1,1,0,0,1,0,0,1,0,0,0
1,filter oli yzze1,60 000,filter oli kendara roda empat guna nyaring kot...,,1,1,0,0,0,0,0,1,0,1,0
2,filter udara avanza vvti,175 000,filter udara kualitas tinggi kendara roda empa...,,1,0,0,0,0,0,0,1,0,1,0
3,service rem depan belakang,300 000,layan awat rem depan belakang kendara roda empat,,0,0,0,0,0,0,0,0,1,0,0
4,tune up injection cleaner,350 000,layan tune up teknologi injection cleaner bers...,,1,1,0,0,1,0,0,1,0,0,0


In [40]:
dataset_2_labeled = dataset_2_labeled.drop(columns=['masalah'])
dataset_2_labeled

,suku_cadang,kategori,biaya,konteks,masalah sistem knalpot,masalah transmisi,masalah suspensi,ganggu listrik,masalah sistem bahan bakar,masalah kemudi,masalah ban,masalah mesin,gagal rem,masalah sistem dingin,rusa aksesoris interior eksterior
0,,,,lihat indikator check engine nyala dashboard m...,0,0,0,0,0,0,0,1,0,0,0
1,,,,saya injak rem dengar suara decit tidak biasa,0,0,0,0,0,0,0,0,1,0,0
2,,,,rasa getar tidak normal kemudi cepat tinggi,0,0,0,0,0,1,1,0,0,0,0
3,,,,mobil stir nya stabil lalu menstater kadang ka...,0,0,0,1,0,1,0,1,0,0,0
4,,,,kaca mobil bisa gerak,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,,,,kondisi ban kendara terlalu licin perlu ganti,0,0,0,0,0,0,1,0,0,0,0
134,,,,mesin kendara keluar suara decit perlu baik,0,0,0,0,0,0,0,1,0,0,0
135,,,,kabel gas kendara terlalu kendor perlu ganti,1,1,0,1,1,0,0,0,0,1,0
136,,,,kondisi ac kendara terlalu dingin perlu setel,1,0,0,0,1,0,0,0,0,1,0


In [41]:
dataset_2_labeled.shape

(121, 15)

## 4.3.5 Integrasi Data

In [42]:
# Menggabungkan data
dataset_gabungan = pd.merge(dataset_1_labeled, dataset_2_labeled, on=LABELS, how='left')
dataset_gabungan.head()

,suku_cadang_x,biaya_x,kategori_x,konteks_x,masalah sistem knalpot,masalah transmisi,masalah suspensi,ganggu listrik,masalah sistem bahan bakar,masalah kemudi,masalah ban,masalah mesin,gagal rem,masalah sistem dingin,rusa aksesoris interior eksterior,suku_cadang_y,kategori_y,biaya_y,konteks_y
0,shell hx 6 10 40 4ltr,420 000,oli mesin kendara roda empat bobot 4 liter fun...,,1,1,0,0,1,0,0,1,0,0,0,NaN,NaN,NaN,NaN
1,filter oli yzze1,60 000,filter oli kendara roda empat guna nyaring kot...,,1,1,0,0,0,0,0,1,0,1,0,NaN,NaN,NaN,NaN
2,filter udara avanza vvti,175 000,filter udara kualitas tinggi kendara roda empa...,,1,0,0,0,0,0,0,1,0,1,0,NaN,NaN,NaN,NaN
3,service rem depan belakang,300 000,layan awat rem depan belakang kendara roda empat,,0,0,0,0,0,0,0,0,1,0,0,,,,saya injak rem dengar suara decit tidak biasa
4,service rem depan belakang,300 000,layan awat rem depan belakang kendara roda empat,,0,0,0,0,0,0,0,0,1,0,0,,,,rem kendara kerja baik perlu rem jarak jauh


In [43]:
dataset_gabungan = dataset_gabungan.drop(columns=['konteks_x', 'suku_cadang_y', 'biaya_y', 'kategori_y'])
dataset_gabungan = dataset_gabungan.rename(columns={'suku_cadang_x': 'suku_cadang', 'biaya_x': 'biaya', 'kategori_x': 'kategori', 'konteks_y': 'konteks'})
dataset_gabungan = dataset_gabungan[['suku_cadang', 'biaya', 'konteks'] + LABELS]

dataset_gabungan.head()

,suku_cadang,biaya,konteks,masalah sistem knalpot,masalah transmisi,masalah suspensi,ganggu listrik,masalah sistem bahan bakar,masalah kemudi,masalah ban,masalah mesin,gagal rem,masalah sistem dingin,rusa aksesoris interior eksterior
0,shell hx 6 10 40 4ltr,420 000,NaN,1,1,0,0,1,0,0,1,0,0,0
1,filter oli yzze1,60 000,NaN,1,1,0,0,0,0,0,1,0,1,0
2,filter udara avanza vvti,175 000,NaN,1,0,0,0,0,0,0,1,0,1,0
3,service rem depan belakang,300 000,saya injak rem dengar suara decit tidak biasa,0,0,0,0,0,0,0,0,1,0,0
4,service rem depan belakang,300 000,rem kendara kerja baik perlu rem jarak jauh,0,0,0,0,0,0,0,0,1,0,0


In [44]:
# Menghapus rows yang tidak memiliki satupun label bernilai 1
has_label_1 = dataset_gabungan.iloc[:, 4:].any(axis=1)
dataset = dataset_gabungan[has_label_1].reset_index(drop=True)
print(dataset.shape)

(688, 14)


In [45]:
# Fill NaN values in konteks column using values from other dataset
dataset['konteks'] = dataset.apply(lambda row: row['konteks'] if pd.notna(row['konteks']) else data_sukucadang.loc[data_sukucadang['Komponen'] == row['suku_cadang'], 'Keterangan'].values[0], axis=1)
dataset.head()

,suku_cadang,biaya,konteks,masalah sistem knalpot,masalah transmisi,masalah suspensi,ganggu listrik,masalah sistem bahan bakar,masalah kemudi,masalah ban,masalah mesin,gagal rem,masalah sistem dingin,rusa aksesoris interior eksterior
0,shell hx 6 10 40 4ltr,420 000,oli mesin kendara roda empat bobot 4 liter fun...,1,1,0,0,1,0,0,1,0,0,0
1,filter oli yzze1,60 000,filter oli kendara roda empat guna nyaring kot...,1,1,0,0,0,0,0,1,0,1,0
2,filter udara avanza vvti,175 000,filter udara kualitas tinggi kendara roda empa...,1,0,0,0,0,0,0,1,0,1,0
3,service rem depan belakang,300 000,saya injak rem dengar suara decit tidak biasa,0,0,0,0,0,0,0,0,1,0,0
4,service rem depan belakang,300 000,rem kendara kerja baik perlu rem jarak jauh,0,0,0,0,0,0,0,0,1,0,0


In [46]:
dataset[LABELS].sum()

masalah sistem knalpot               104
masalah transmisi                    218
masalah suspensi                      31
ganggu listrik                       111
masalah sistem bahan bakar            74
masalah kemudi                        31
masalah ban                           70
masalah mesin                        163
gagal rem                             52
masalah sistem dingin                 88
rusa aksesoris interior eksterior    122
dtype: int64

In [47]:
dataset[LABELS].sum().sum()

1064

In [48]:
dataset.to_csv('dataset.csv', index=False)

# 4.4 Modeling

In [ ]:
!pip install transformers tensorflow

In [50]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from sklearn.model_selection import train_test_split

## 4.4.1 Skenario Modeling

### Balancing Oversampling

In [99]:
from sklearn.utils import resample
df = dataset.copy()

labels_summary = dataset.iloc[:, 3:].sum()
labels_summary

max_count = labels_summary.max()

balanced_df = pd.DataFrame(columns=df.columns)
for label in labels_summary.index:
  label_count = labels_summary[label]
  if label_count < max_count:
    label_df = df[df[label] == 1]
    oversampled_df = resample(label_df, replace=True, n_samples=max_count, random_state=42)
    balanced_df = pd.concat([balanced_df, oversampled_df], ignore_index=True)
  else:
    label_df = df[df[label == 1]].sample(n=max_count, random_state=42)
    balanced_df = pd.concat([balanced_df, label_df], ignore_index=True)

balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)
balanced_df

KeyError: ignored

### Balancing SMOTE

In [57]:
!pip install imbalanced-learn

In [66]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority')
X_sm, y_sm = smote.fit_resample(X, y)
X_sm, y_sm

ValueError: ignored

### Tokenisasi

In [112]:
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf

In [113]:
BERT_NAME = 'indobenchmark/indobert-lite-base-p1'
bert_tokenizer = BertTokenizer.from_pretrained(BERT_NAME)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'AlbertTokenizerFast'. 
The class this function is called from is 'BertTokenizer'.


In [123]:
def tokenizing(tokenizer, max_len, sentences):
  input_ids, attention_mask = [], []
  for sentence in sentences:
    # proses preprocessing sebelumnya akan dimasukkan disini
    tokenized = tokenizer.encode_plus(
      sentence,
      add_special_tokens=True,
      max_length=max_len,
      padding='max_length',
      truncation=True,
      return_attention_mask=True,
      return_token_type_ids=False,
      return_tensors='tf'
    )

    input_ids.append(tokenized['input_ids'])
    attention_mask.append(tokenized['attention_mask'])

  return {
    'input_ids': tf.convert_to_tensor(np.asarray(input_ids).squeeze(), dtype=tf.int32),
    'attention_mask': tf.convert_to_tensor(np.asarray(attention_mask).squeeze(), dtype=tf.int32)
  }

In [124]:
def convert_id_to_token(tokenizer, input_ids):
  input_length = len(input_ids)
  return [tokenizer.convert_ids_to_tokens(hasil_tokenisasi['input_ids'][i][:20]) for i in range(input_length)]

In [133]:
def preprocessing(tokenizer, max_len, sentences):
  input_ids, attention_mask = [], []
  for sentence in sentences:
    input = casefolding(sentence)
    input = stemming(input)
    input = stopwordremove(input)
    tokenized = tokenizer.encode_plus(
      input,
      add_special_tokens=True,
      max_length=max_len,
      padding='max_length',
      truncation=True,
      return_attention_mask=True,
      return_token_type_ids=False,
      return_tensors='tf'
    )
    input_ids.append(tokenized['input_ids'])
    attention_mask.append(tokenized['attention_mask'])

  return {
    'input_ids': tf.convert_to_tensor(np.asarray(input_ids).squeeze(), dtype=tf.int32),
    'attention_mask': tf.convert_to_tensor(np.asarray(attention_mask).squeeze(), dtype=tf.int32)
  }

In [126]:
hasil_tokenisasi = tokenizing(bert_tokenizer, 256, hasil_stopword_remove)

print('shape = ', hasil_tokenisasi['input_ids'].shape, hasil_tokenisasi['attention_mask'].shape)
print('\ninput_ids = ', hasil_tokenisasi['input_ids'][0][:20])
print('attention_mask = ', hasil_tokenisasi['attention_mask'][0][:20])


hasil_token = convert_id_to_token(bert_tokenizer, hasil_tokenisasi['input_ids'])
print('\ntoken = ', hasil_token)

shape =  (3, 256) (3, 256)

input_ids =  tf.Tensor(
[    2  1173  5659  1980  6637 14906 26259   895     3     0     0     0
     0     0     0     0     0     0     0     0], shape=(20,), dtype=int32)
attention_mask =  tf.Tensor([1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0], shape=(20,), dtype=int32)

token =  [['[CLS]', 'lihat', 'indikator', 'check', 'engine', 'nyala', 'dashboard', 'mobil', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]'], ['[CLS]', 'saya', 'inj', '##ak', 'rem', 'dengar', 'suara', 'dec', '##it', 'tidak', 'biasa', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]'], ['[CLS]', 'rasa', 'getar', 'tidak', 'normal', 'kemudi', 'cepat', 'tinggi', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']]


### Split Dataset

In [109]:
MAX_LEN=128
TRAIN_BATCH_SIZE=32
TEST_BATCH_SIZE=32
TRAIN_SPLIT=0.8
TEST_SPLIT=0.2

In [116]:
X_train, X_test, y_train, y_test = train_test_split(
  dataset['konteks'],
  dataset[LABELS],
  test_size=TEST_SPLIT,
  shuffle=True,
  random_state=42
)

X_train = tokenizing(bert_tokenizer, MAX_LEN, X_train)
X_test = tokenizing(bert_tokenizer, MAX_LEN, X_test)

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(TRAIN_BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(TEST_BATCH_SIZE)

## 4.4.2 Pembangunan Model

In [117]:
bert_model = TFBertModel.from_pretrained(BERT_NAME)

You are using a model of type albert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Some layers from the model checkpoint at indobenchmark/indobert-lite-base-p1 were not used when initializing TFBertModel: ['albert', 'sop_classifier', 'predictions']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertModel were not initialized from the model checkpoint at indobenchmark/indobert-lite-base-p1 and are newly initialized: ['bert']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [110]:
EPOCHS=10
LEARNING_RATE=2e-05

In [118]:
def classification_model(bert_encoder, num_labels, max_len, learning_rate):
  input_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name='input_ids')
  attention_mask = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name='attention_mask')

  encoding_layer = bert_encoder(input_ids, attention_mask)[0]

  l = tf.keras.layers.GlobalAveragePooling1D(name='pooling_layer')(encoding_layer)
  l = tf.keras.layers.Dropout(0.1, name='dropout_layer')(l)
  l = tf.keras.layers.Dense(num_labels, activation='sigmoid', name='output_layer')(l)

  model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=[l])

  OPTIMIZER = tf.keras.optimizers.Adam(learning_rate=learning_rate)
  LOSS = tf.keras.losses.BinaryCrossentropy()
  METRICS = [tf.keras.metrics.BinaryAccuracy(name='accuracy')]

  model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=METRICS)

  return model

model = classification_model(
    bert_encoder=bert_model,
    num_labels=len(LABELS),
    max_len=MAX_LEN,
    learning_rate=LEARNING_RATE,
)

### Custom Dataset and Classifier

In [ ]:
class CustomDataset(tf.keras.utils.Sequence):
  def __init__(self, data, tokenizer, max_len):
    self.data = data.reset_index(drop=True)
    self.tokenizer = tokenizer
    self.max_len = max_len
    self.indexes = np.arrange(len(self.data))

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    row = self.data.iloc[index]

    konteks = row['konteks'].tolist()
    sparepart = row['sparepart'].tolist()
    price = row['price'].tolist()
    labels = row.drop(['konteks', 'sparepart', 'price'], axis=1)

    input = casefolding(konteks)
    input = stemming(input)
    input = stopwordremove(input)

    tokenized = self.tokenizer.encode_plus(
        input,
        add_special_tokens=True,
        max_length=self.max_len,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_token_type_ids=False,
        return_tensors='tf'
    )

    input_ids = tf.convert_to_tensor(np.asarray(tokenized['input_ids']).squeeze(), dtype=tf.int32)
    attention_mask = tf.convert_to_tensor(np.asarray(tokenized['attention_mask']).squeeze(), dtype=tf.int32)

    return {
      'input_ids': input_ids,
      'attention_mask': attention_mask,
      'labels': labels,
      'sparepart': sparepart,
      'price': price
    }


In [ ]:
class ClassificationModelWithRecommendation(tf.keras.Model):
  def __init__(self, bert_encoder, num_labels, num_spare_parts, max_len, learning_rate):
    super(ClassificationModelWithRecommendation, self).__init__()
    self.bert_encoder = bert_encoder
    self.num_labels = num_labels
    self.num_spare_parts = num_spare_parts
    self.max_len = max_len
    self.learning_rate = learning_rate

    # Init classification submodel
    self.classification_model = self.build_classification_model()

    # Init spareparts prediction submodel
    self.spare_part_model = self.build_spare_part_model()

    # Init estimated price submodel
    self.price_estimation_model = self.build_price_estimation_model()

  def build_classification_model(self):
    input_ids = tf.keras.layers.Input(shape=(self.max_len,), dtype=tf.int32, name='input_ids')
    attention_mask = tf.keras.layers.Input(shape=(self.max_len,), dtype=tf.int32, name='attention_mask')

    encoding_layer = self.bert_encoder(input_ids, attention_mask)[0]

    l = tf.keras.layers.GlobalAveragePooling1D(name='poolling_layer')(encoding_layer)
    l = tf.keras.layers.Dropout(0.1, name='dropout_layer')(l)
    l = tf.keras.layers.Dense(self.num_labels, action='sigmoid', name='output_layer')(l)

    model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=[l])

    OPTIMIZER = tf.keras.optimizers.Adam(learning_rate=self.learning_rate)
    LOSS = tf.keras.losses.BinaryCrossentropy()
    METRICS = [tf.keras.metrics.BinaryAccuracy(name='accuracy')]

    model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=METRICS)

    return model

  def build_spare_part_model(self):
    spare_part_input = tf.keras.layers.Input(shape=(self.num_labels,), name='spare_part_input')
    spare_part_output = tf.keras.layers.Dense(self.num_spare_parts, activation='softmax', name='spare_part_output')(spare_part_input)
    model = tf.keras.Model(inputs=spare_part_input, outputs=spare_part_output)
    return model

  def build_price_estimation_model(self):
    price_input = tf.keras.layers.Input(shape=(self.num_labels,), name='price_input')
    price_output = tf.keras.layers.Dense(self.num_spare_parts, activation='linear', name='price_output')(price_input)
    model = tf.keras.Model(inputs=price_input, outputs=price_output)
    return model

  def call(self, inputs):
    classification_output = self.classification_model(inputs)
    spare_part_probs = self.spare_part_model(classification_output)
    price_estimations = self.price_estimation_model(classification_output)
    return classification_output, spare_part_probs, price_estimations


model_with_recommendation = ClassificationModelWithRecommendation(
    bert_encoder=bert_model,
    num_labels=len(LABELS),
    num_spare_parts=len(),
    max_len=MAX_LEN,
    learning_rate=LEARNING_RATE
)

### Model Summary amd Training

In [119]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 128)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  109081344   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 128,                                           

In [120]:
model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=test_dataset,
)

Epoch 1/10


18/18 [==============================] - 930s 49s/step - loss: 0.4882 - accuracy: 0.8321 - val_loss: 0.4368 - val_accuracy: 0.8491
Epoch 2/10
18/18 [==============================] - 859s 48s/step - loss: 0.3340 - accuracy: 0.8754 - val_loss: 0.3128 - val_accuracy: 0.8808
Epoch 3/10
18/18 [==============================] - 859s 48s/step - loss: 0.2421 - accuracy: 0.9164 - val_loss: 0.2277 - val_accuracy: 0.9051
Epoch 4/10
18/18 [==============================] - 856s 48s/step - loss: 0.1689 - accuracy: 0.9375 - val_loss: 0.1796 - val_accuracy: 0.9269
Epoch 5/10
18/18 [==============================] - 857s 48s/step - loss: 0.1108 - accuracy: 0.9603 - val_loss: 0.1239 - val_accuracy: 0.9526
Epoch 6/10
18/18 [==============================] - 859s 48s/step - loss: 0.0760 - accuracy: 0.9740 - val_loss: 0.0885 - val_accuracy: 0.9638
Epoch 7/10
18/18 [==============================] - 864s 48s/step - loss: 0.0518 - accuracy: 0.9813 - val_loss: 0.0660 - val_accuracy: 0.9743
Epoch 8/10
18/18 

# 4.5 Model Evaluation

In [122]:
model.evaluate(test_dataset)

5/5 [==============================] - 86s 16s/step - loss: 0.0718 - accuracy: 0.9743


[0.07176430523395538, 0.9743083119392395]

In [134]:
inputs = ['AC mobil tidak dingin', 'Rem pada mobil tidak bekerja', 'Kaca spion rusak harus diganti']
tokenized = preprocessing(bert_tokenizer, MAX_LEN, inputs)
predictions = model.predict(tokenized)

# Define the threshold
threshold = 0.5

# Apply the threshold to the predictions
binary_predictions = np.where(predictions > threshold, 1, 0)

# Print the binary predictions
print(LABELS)
print(binary_predictions)

1/1 [==============================] - 6s 6s/step
['masalah sistem knalpot', 'masalah transmisi', 'masalah suspensi', 'ganggu listrik', 'masalah sistem bahan bakar', 'masalah kemudi', 'masalah ban', 'masalah mesin', 'gagal rem', 'masalah sistem dingin', 'rusa aksesoris interior eksterior']
[[1 0 0 0 1 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 1]]


In [135]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

# Assuming you have predictions and ground truth labels
predictions = model.predict(test_dataset)  # Get model predictions
threshold = 0.5  # Set threshold for label prediction

# Convert probability predictions to binary predictions
binary_predictions = np.where(predictions >= threshold, 1, 0)

# Compute precision, recall, and F1-score for each label
label_precisions = precision_score(y_test, binary_predictions, average=None)
label_recalls = recall_score(y_test, binary_predictions, average=None)
label_f1_scores = f1_score(y_test, binary_predictions, average=None)

# Print accuracy metrics for each label
for label, precision, recall, f1_score in zip(LABELS, label_precisions, label_recalls, label_f1_scores):
    print(f"Label: {label}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-score: {f1_score}")
    print()


5/5 [==============================] - 66s 12s/step
Label: masalah sistem knalpot
Precision: 0.8125
Recall: 1.0
F1-score: 0.896551724137931

Label: masalah transmisi
Precision: 0.9782608695652174
Recall: 1.0
F1-score: 0.989010989010989

Label: masalah suspensi
Precision: 0.9
Recall: 1.0
F1-score: 0.9473684210526316

Label: ganggu listrik
Precision: 0.92
Recall: 1.0
F1-score: 0.9583333333333334

Label: masalah sistem bahan bakar
Precision: 0.5454545454545454
Recall: 1.0
F1-score: 0.7058823529411764

Label: masalah kemudi
Precision: 0.9090909090909091
Recall: 1.0
F1-score: 0.9523809523809523

Label: masalah ban
Precision: 0.95
Recall: 1.0
F1-score: 0.9743589743589743

Label: masalah mesin
Precision: 0.9545454545454546
Recall: 1.0
F1-score: 0.9767441860465117

Label: gagal rem
Precision: 1.0
Recall: 1.0
F1-score: 1.0

Label: masalah sistem dingin
Precision: 0.6774193548387096
Recall: 1.0
F1-score: 0.8076923076923077

Label: rusa aksesoris interior eksterior
Precision: 1.0
Recall: 1.0
F1-s

# 4.6 Deployment

In [136]:
!rm model_output/klasifikasi.h5
model.save('model_output/klasifikasi.h5')

In [137]:
!cp model_output/klasifikasi.h5 '/content/drive/MyDrive/Kuliah/TA Akhdan Musyaffa Firdaus/Bimbingan - Akhdan Musyaffa Firdaus/Produk/Model'

In [139]:
loaded_model = tf.keras.models.load_model('/content/drive/MyDrive/Kuliah/TA Akhdan Musyaffa Firdaus/Bimbingan - Akhdan Musyaffa Firdaus/Produk/Model/klasifikasi.h5', custom_objects={'TFBertModel': TFBertModel})

In [140]:
inputs = ['mesin tidak bisa menyala', 'AC mobil tidak dingin', 'Mesin bersuara kasar', 'ban bocor dan harus diganti dengan yang baru', 'asap knalpot berwarna putih', 'kaca spion pecah dan harus diganti', 'lampu sen kendaraan tidak berfungsi']
tokenized = preprocessing(bert_tokenizer, MAX_LEN, inputs)
predictions = loaded_model.predict(tokenized)

# Define the threshold
threshold = 0.5

# Apply the threshold to the predictions
binary_predictions = np.where(predictions > threshold, 1, 0)

# Print the binary predictions
print(LABELS)
print(binary_predictions)

1/1 [==============================] - 8s 8s/step
['masalah sistem knalpot', 'masalah transmisi', 'masalah suspensi', 'ganggu listrik', 'masalah sistem bahan bakar', 'masalah kemudi', 'masalah ban', 'masalah mesin', 'gagal rem', 'masalah sistem dingin', 'rusa aksesoris interior eksterior']
[[0 0 0 0 0 0 0 1 0 0 0]
 [1 0 0 0 1 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 1 0 0 0]
 [1 0 0 0 1 0 1 0 0 1 0]
 [1 0 0 0 1 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 1 1 0 0 0 0 1 0]]


## Implementasi Rumus menentukan tingkat kerusakan

In [ ]:
LABEL_WEIGHTS = dict.fromkeys(label_kerusakan, 0)

def measure_severity(val):
  treshold = 0.5

  NUMBER_LABEL_DETECTED = np.sum(val > treshold)
  LABEL_RESULTS = {}

  for label, count in zip(LABEL_WEIGHTS.keys(), val):
    if count > treshold:
      LABEL_RESULTS[label] = count

  category_weights = [LABEL_WEIGHTS[label] * count for label, count in LABEL_RESULTS.items()]
  severity_score = sum(category_weights) / NUMBER_LABEL_DETECTED

  return severity_score